In [64]:
#classification implementation code --- to be cleaned up and pasted into Technical Report

import importlib
from tabulate import tabulate

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation


In [60]:
#loading data
stroke_data = MyPyTable()
stroke_data.load_from_file("input_data/stroke-data-downsized.csv")
print("attribute names:", stroke_data.column_names)
print("len before removing missing vals", len(stroke_data.data))

#code to remove rows w/missing values (use only once relevant attributes are decided)
#stroke_data.remove_rows_with_missing_values()
#stroke_data.remove_rows_with_missing_values_by_col()
#print("len after removing missing vals", len(stroke_data.data))

#code to group data by class
stroke_classes, stroke_data_by_class = myutils.group_by(stroke_data.data, stroke_data.column_names, "stroke")
print("classes:", stroke_classes)
for partition in stroke_data_by_class:
    print("num instances of class", partition[0][-1], ":", len(partition))



attribute names: ['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']
len before removing missing vals 1000
classes: [0.0, 1.0]
num instances of class 0.0 : 791
num instances of class 1.0 : 209


In [67]:
#make classifiers - KNN, NB, DT, RF
knn_clf = MyKNeighborsClassifier(n_neighbors=5)
nb_clf = MyNaiveBayesClassifier()
dt_clf = MyDecisionTreeClassifier()
#TODO: random forest

#TODO clean stroke data for classification- discretize, convert nominal to numeric
print(stroke_data.column_names, "\n", stroke_data.data[0])
stroke_data_discretized = myutils.clean_attributes_for_stroke_classification(stroke_data)
print(stroke_data_discretized.column_names, "\n", stroke_data_discretized.data[0])
#save discretized to file
stroke_data_discretized.save_to_file("input_data/stroke-data-discretized.csv")


#Create xtrain/ytrains (TODO change to stratified k fold later)
X_train = [inst[:-1] for inst in stroke_data.data]
y_train = [inst[-1] for inst in stroke_data.data]

#TODO fit and predict classifiers

#TODO Display results

['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke'] 
 [9046.0, 'Male', 67.0, 0.0, 1.0, 'Yes', 'Private', 'Urban', 228.69, 36.6, 'formerly smoked', 1.0]
['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke'] 
 ['Male', 6.0, 0.0, 1.0, 'Yes', 'Private', 'Urban', 5, 3.0, 'formerly smoked', 1.0]
